In [14]:
import os

# Caminho base
caminho_base = 'Dados'

# Listar apenas subpastas
subpastas = [nome for nome in os.listdir(caminho_base) 
             if os.path.isdir(os.path.join(caminho_base, nome))]

print(subpastas)

['.ipynb_checkpoints', '2019T1', '2022T1', '2024T1', '2024T2', '2024T3', '2024T4', '2025T1']


In [15]:
import pandas as pd

ibge = pd.read_excel('municipios_SC_completo_2022.xlsx')
display(ibge)

,Município,População (Censo 2022)
0,Joinville,616317
1,Florianópolis,537211
2,Blumenau,361261
3,São José,270299
4,Itajaí,264054
...,...,...
290,Jardinópolis,1776
291,Lajeado Grande,1702
292,Presidente Castello Branco,1689
293,Barra Bonita,1668


In [1]:
import pandas as pd
import os

# Caminho base
caminho_base = 'Dados'

# Listar apenas subpastas
subpastas = [nome for nome in os.listdir(caminho_base) 
             if os.path.isdir(os.path.join(caminho_base, nome))]

def carregar_excel_flexivel(caminho_base, nome_arquivo_base):
    for ext in ['.xlsx', '.xls']:
        caminho_completo = os.path.join(caminho_base, nome_arquivo_base + ext)
        if os.path.exists(caminho_completo):
            return pd.read_excel(caminho_completo)
    raise FileNotFoundError(f'Arquivo {nome_arquivo_base}.xls[x] não encontrado em {caminho_base}')

for subpasta in subpastas[2:]:
    caminho_base = f'Dados/{subpasta}'
    inf_municipais = pd.read_excel('Informacoes_Municipios.xlsx')
    municipios_validos = inf_municipais['NM_MUN'].unique()
    meio_de_hospedagem = carregar_excel_flexivel(caminho_base, f'{subpasta} - meio-de-hospedagem')
    empreendimento_lazer = carregar_excel_flexivel(caminho_base, f'{subpasta} - parque-aquatico-e-empreendimento-de-lazer')
    parque_tematico = carregar_excel_flexivel(caminho_base, f'{subpasta} - parque-tematico')
    restaurantes = carregar_excel_flexivel(caminho_base, f'{subpasta} - restaurante-cafeteria-bar-e-similares')

    dataframes = [meio_de_hospedagem, empreendimento_lazer, parque_tematico, restaurantes]

    for df in dataframes:
        # Corrigir a UF de municípios fora de SC
        df.loc[df['Município'] == 'Balneário Pinhal', 'UF'] = 'RS'
        df.loc[df['Município'] == 'Anhangüera', 'UF'] = 'GO'
        df.loc[df['Município'] == 'Ubatuba', 'UF'] = 'SP'
        df.loc[df['Município'] == 'Baliza', 'UF'] = 'GO'
    
        # Corrigir nomes de municípios (normalização)
        df.loc[df['Município'] == 'Grão Pará', 'Município'] = 'Grão-Pará'
        df.loc[df['Município'] == "Herval D'Oeste", 'Município'] = "Herval d'Oeste"
        df.loc[df['Município'] == 'Guatambú', 'Município'] = 'Guatambu'
        df.loc[df['Município'] == 'Bocaína do Sul', 'Município'] = 'Bocaina do Sul'

    # 1. Filtra todos os DataFrames com base nessa lista
    df1 = meio_de_hospedagem[meio_de_hospedagem['Município'].isin(municipios_validos)]
    df2 = empreendimento_lazer[empreendimento_lazer['Município'].isin(municipios_validos)]
    df3 = parque_tematico[parque_tematico['Município'].isin(municipios_validos)]
    df4 = restaurantes[restaurantes['Município'].isin(municipios_validos)]
    
    todos_municipios = sorted(set(df1['Município']) |
                              set(df2['Município']) |
                              set(df3['Município']) |
                              set(df4['Município']) |
                              set(inf_municipais['NM_MUN']))
    
    # 3. Cria o dicionário
    dados_por_municipio = {}
    for municipio in todos_municipios:
        dados_por_municipio[municipio] = {
            'meio_de_hospedagem': df1[df1['Município'] == municipio],
            'empreendimento_lazer': df2[df2['Município'] == municipio],
            'parque_tematico': df3[df3['Município'] == municipio],
            'restaurantes': df4[df4['Município'] == municipio],
            'inf_municipais': inf_municipais[inf_municipais['NM_MUN'] == municipio]
        }

    #criação do dataframe com as informações desejadas
    dados = []
    
    for municipio, info in dados_por_municipio.items():
        nome = municipio
    
        # População
        df_inf_mun = info['inf_municipais']
        populacao = df_inf_mun.iloc[0, 19] if not df_inf_mun.empty else None  
        dist_aero = df_inf_mun.iloc[0, 18] if not df_inf_mun.empty else None #distancia aero
        nome_aero = df_inf_mun.iloc[0, 17] if not df_inf_mun.empty else None #nome aero
        icms = df_inf_mun.iloc[0, 20] if not df_inf_mun.empty else None #icms
        
        area_mun = df_inf_mun.iloc[0, 15] if not df_inf_mun.empty else None
        regiao_turismo = df_inf_mun.iloc[0, 16] if not df_inf_mun.empty else None
        cod_uf = df_inf_mun.iloc[0, 7] if not df_inf_mun.empty else None
        cod_mun = df_inf_mun.iloc[0, 1] if not df_inf_mun.empty else None
        
        #RESTAURANTES
        # Total de restaurantes
        total_restaurantes = len(info['restaurantes'])
    
        #PARQUE TEMATICOS
        # Total de parques temáticos
        total_parques = len(info['parque_tematico'])
    
        # Soma da área total de parque tematico
        df_parque_tematico = info['parque_tematico']
        if 'Área Total do Empreendimento' in df_parque_tematico.columns:
            area_total_parque_tematico = (
            df_parque_tematico['Área Total do Empreendimento']
            .replace('-', None)  # Substitui o traço por None
            .astype(float)
            .sum(min_count=1)
        )
            col_area = df_parque_tematico['Área Total do Empreendimento'].replace(['-', 'ND', 'n/d', 'NA', ''], None)
            parques_sem_area = col_area.isna().sum()
        else:
            area_total_parque_tematico = None
            parques_sem_area = 0
        #EMPREENDIMENTOS DE LAZER
        # Total
        total_empreendimentos_lazer = len(info['empreendimento_lazer'])
        
        #Soma 'Área Total Construída(m²)'
        df_empreendimento_lazer = info['empreendimento_lazer']
        if 'Área Total Construída(m²)' in df_empreendimento_lazer.columns:
            area_total_empreendimento_lazer = (
                df_empreendimento_lazer['Área Total Construída(m²)']
                .replace('-', None)
                .astype(float)
                .sum(min_count=1)
            )
            col_area = df_empreendimento_lazer['Área Total Construída(m²)'].replace(['-', 'ND', 'n/d', 'NA', ''], None)
            empreendimentos_lazer_sem_area = col_area.isna().sum()
        else:
            area_total_empreendimento_lazer = None
            empreendimentos_lazer_sem_area = 0
    
        #MEIO DE HOSPEDAGEM
        # Total
        total_hospedagem = len(info['meio_de_hospedagem'])
        
        # Unidades Habitacionais
        df_hospedagem = info['meio_de_hospedagem']
        if 'Unidade Habitacionais' in df_hospedagem.columns:
            total_unidades_habitacionais = (
                df_hospedagem['Unidade Habitacionais']
                .replace('-', None)
                .astype(float)
                .sum(min_count=1)
        )
            col_uni_hab = df_hospedagem['Unidade Habitacionais'].replace(['-', 'ND', 'n/d', 'NA', ''], None)
            hosp_sem_uni_hab = col_uni_hab.isna().sum()
        else:
            total_unidades_habitacionais = 0
            col_uni_hab = df_hospedagem['Unidade Habitacionais'].replace(['-', 'ND', 'n/d', 'NA', ''], None)
            hosp_sem_uni_hab = col_uni_hab.isna().sum()
    
        # Leitos
        if 'Leitos' in df_hospedagem.columns:
            total_leitos = (
                df_hospedagem['Leitos']
                .replace('-', None)
                .astype(float)
                .sum(min_count=1)
            )
            col_leitos = df_hospedagem['Leitos'].replace(['-', 'ND', 'n/d', 'NA', ''], None)
            hosp_sem_leitos = col_leitos.isna().sum()
        else:
            total_leitos = 0
            col_leitos = df_hospedagem['Leitos'].replace(['-', 'ND', 'n/d', 'NA', ''], None)
            hosp_sem_leitos = col_leitos.isna().sum()
    
        # Média de UH por hospedagem
        media_uh = total_unidades_habitacionais/total_hospedagem
        
        # Média de Leitos por Unidade Habitacional
        media_leitos = total_leitos/total_unidades_habitacionais
        
        # Acumula os dados
        dados.append({
            'Nome_Municipio': nome,
            'Regiao_Turistica': regiao_turismo,
            'Distancia_Aero_Km': dist_aero,
            'Nome_Aeroporto': nome_aero,
            'Populacao': populacao,
            'Area_Mun_Km2': area_mun,
            'ICMS_1S2025': icms,
            'Cod_UF': cod_uf,
            'Cod_Mun': cod_mun,
            'Total_Restaurantes': total_restaurantes,
            'Total_Parques_Tematicos': total_parques,
            'Area_Total_Parques': area_total_parque_tematico,
            'Parques_Sem_Area': parques_sem_area,
            'Total_Empreendimentos_Lazer': total_empreendimentos_lazer,
            'Area_Total_Empreendimento_Lazer': area_total_empreendimento_lazer,
            'Empreendimentos_Lazer_Sem_Area': empreendimentos_lazer_sem_area,
            'Total_Hospedagem': total_hospedagem,
            'Unidades_Habitacionais': total_unidades_habitacionais,
            'Hospedagem_Sem_Dados_Unidades_Habitacionais': hosp_sem_uni_hab,
            'Media_Unidades_Habitacionais_por_Hospedagem': media_uh,
            'Leitos': total_leitos,
            'Hospedagem_Sem_Dados_Leitos': hosp_sem_leitos,
            'Media_Leitos_por_UH': media_leitos
        })
    
    informacoes_municipio = pd.DataFrame(dados)
    informacoes_municipio['Restaurantes_por_mil_hab'] = (informacoes_municipio['Total_Restaurantes'] / informacoes_municipio['Populacao'] * 1000)
    informacoes_municipio['Hospedagem_por_mil_hab'] = (informacoes_municipio['Total_Hospedagem'] / informacoes_municipio['Populacao'] * 1000)
    informacoes_municipio['Leitos_por_mil_hab'] = (informacoes_municipio['Leitos'] / informacoes_municipio['Populacao'] * 1000)
    informacoes_municipio.to_excel(f'v1 - {subpasta} - Informações por Município.xlsx', index=False)

In [35]:
print(type(df1))
print(type(df2))
print(type(df3))
print(type(df4))
print(type(populacao))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'NoneType'>


In [7]:
print(inf_municipais.columns)

Index(['Unnamed: 0', 'CD_MUN', 'NM_MUN', 'CD_RGI', 'NM_RGI', 'CD_RGINT',
       'NM_RGINT', 'CD_UF', 'NM_UF', 'SIGLA_UF', 'CD_REGIA', 'NM_REGIA',
       'SIGLA_RG', 'CD_CONCU', 'NM_CONCU', 'AREA_KM2', 'DIV_TUR', 'HubName',
       'HubDist', 'POP. TOTAL', 'ICMS 1S2025'],
      dtype='object')


In [11]:
informacoes_municipio.columns

Index(['Nome_Municipio', 'Regiao_Turistica', 'Distancia_Aero_Km',
       'Nome_Aeroporto', 'Populacao', 'Area_Mun_Km2', 'ICMS_1S2025', 'Cod_UF',
       'Cod_Mun', 'Total_Restaurantes', 'Total_Parques_Tematicos',
       'Area_Total_Parques', 'Parques_Sem_Area', 'Total_Empreendimentos_Lazer',
       'Area_Total_Empreendimento_Lazer', 'Empreendimentos_Lazer_Sem_Area',
       'Total_Hospedagem', 'Unidades_Habitacionais',
       'Hospedagem_Sem_Dados_Unidades_Habitacionais',
       'Media_Unidades_Habitacionais_por_Hospedagem', 'Leitos',
       'Hospedagem_Sem_Dados_Leitos', 'Media_Leitos_por_UH'],
      dtype='object')

## RASCUNHOS

In [11]:
#imporação da base de dados
import pandas as pd

meio_de_hospedagem = pd.read_excel('meio-de-hospedagem.xlsx')
empreendimento_lazer = pd.read_excel('parque-aquatico-e-empreendimento-de-lazer.xlsx')
parque_tematico = pd.read_excel('parque-tematico.xlsx')
restaurantes = pd.read_excel('restaurante-cafeteria-bar-e-similares.xlsx')
populacao = pd.read_excel('municipios_SC_completo_2022.xlsx')

# Lista com os dataframes a serem corrigidos
dataframes = [meio_de_hospedagem, empreendimento_lazer, parque_tematico, restaurantes]

for df in dataframes:
    # Corrigir a UF de municípios fora de SC
    df.loc[df['Município'] == 'Balneário Pinhal', 'UF'] = 'RS'
    df.loc[df['Município'] == 'Anhangüera', 'UF'] = 'GO'
    df.loc[df['Município'] == 'Ubatuba', 'UF'] = 'SP'
    df.loc[df['Município'] == 'Baliza', 'UF'] = 'GO'

    # Corrigir nomes de municípios (normalização)
    df.loc[df['Município'] == 'Grão Pará', 'Município'] = 'Grão-Pará'
    df.loc[df['Município'] == "Herval D'Oeste", 'Município'] = "Herval d'Oeste"
    df.loc[df['Município'] == 'Guatambú', 'Município'] = 'Guatambu'
    df.loc[df['Município'] == 'Bocaína do Sul', 'Município'] = 'Bocaina do Sul'

In [65]:
#criação do dicionário geral (dados_por_municipio)
#é possível depois melhorar o código -> funcionalizar

# 1. Filtra todos os DataFrames para 'SC'
df1 = meio_de_hospedagem[meio_de_hospedagem['UF'] == 'SC']
df2 = empreendimento_lazer[empreendimento_lazer['UF'] == 'SC']
df3 = parque_tematico[parque_tematico['UF'] == 'SC']
df4 = restaurantes[restaurantes['UF'] == 'SC']

# 2. Coleta todos os municípios únicos da UF 'SC'
todos_municipios = sorted(set(df1['Município']) |
                          set(df2['Município']) |
                          set(df3['Município']) |
                          set(df4['Município']) |
                          set(populacao['Município']))

# 3. Cria o dicionário
dados_por_municipio = {}

for municipio in todos_municipios:
    dados_por_municipio[municipio] = {
        'meio_de_hospedagem': df1[df1['Município'] == municipio],
        'empreendimento_lazer': df2[df2['Município'] == municipio],
        'parque_tematico': df3[df3['Município'] == municipio],
        'restaurantes': df4[df4['Município'] == municipio],
        'populacao': populacao[populacao['Município'] == municipio]
    }

In [10]:
#criação do dataframe com as informações desejadas
dados = []

for municipio, info in dados_por_municipio.items():
    nome = municipio

    # População
    df_pop = info['populacao']
    populacao = df_pop.iloc[0, 1] if not df_pop.empty else None  # Considerando que a 2ª coluna é a população

    #RESTAURANTES
    # Total de restaurantes
    total_restaurantes = len(info['restaurantes'])

    #PARQUE TEMATICOS
    # Total de parques temáticos
    total_parques = len(info['parque_tematico'])

    # Soma da área total de parque tematico
    df_parque_tematico = info['parque_tematico']
    if 'Área Total do Empreendimento' in df_parque_tematico.columns:
        area_total_parque_tematico = (
        df_parque_tematico['Área Total do Empreendimento']
        .replace('-', None)  # Substitui o traço por None
        .astype(float)
        .sum(min_count=1)
    )
        col_area = df_parque_tematico['Área Total do Empreendimento'].replace(['-', 'ND', 'n/d', 'NA', ''], None)
        parques_sem_area = col_area.isna().sum()
    else:
        area_total_parque_tematico = None
        parques_sem_area = 0
    #EMPREENDIMENTOS DE LAZER
    # Total
    total_empreendimentos_lazer = len(info['empreendimento_lazer'])
    
    #Soma 'Área Total Construída(m²)'
    df_empreendimento_lazer = info['empreendimento_lazer']
    if 'Área Total Construída(m²)' in df_empreendimento_lazer.columns:
        area_total_empreendimento_lazer = (
            df_empreendimento_lazer['Área Total Construída(m²)']
            .replace('-', None)
            .astype(float)
            .sum(min_count=1)
        )
        col_area = df_empreendimento_lazer['Área Total Construída(m²)'].replace(['-', 'ND', 'n/d', 'NA', ''], None)
        empreendimentos_lazer_sem_area = col_area.isna().sum()
    else:
        area_total_empreendimento_lazer = None
        empreendimentos_lazer_sem_area = 0

    #MEIO DE HOSPEDAGEM
    # Total
    total_hospedagem = len(info['meio_de_hospedagem'])
    
    # Unidades Habitacionais
    df_hospedagem = info['meio_de_hospedagem']
    if 'Unidade Habitacionais' in df_hospedagem.columns:
        total_unidades_habitacionais = (
            df_hospedagem['Unidade Habitacionais']
            .replace('-', None)
            .astype(float)
            .sum(min_count=1)
    )
        col_uni_hab = df_hospedagem['Unidade Habitacionais'].replace(['-', 'ND', 'n/d', 'NA', ''], None)
        hosp_sem_uni_hab = col_uni_hab.isna().sum()
    else:
        total_unidades_habitacionais = 0
        col_uni_hab = df_hospedagem['Unidade Habitacionais'].replace(['-', 'ND', 'n/d', 'NA', ''], None)
        hosp_sem_uni_hab = col_uni_hab.isna().sum()

    # Leitos
    if 'Leitos' in df_hospedagem.columns:
        total_leitos = (
            df_hospedagem['Leitos']
            .replace('-', None)
            .astype(float)
            .sum(min_count=1)
        )
        col_leitos = df_hospedagem['Leitos'].replace(['-', 'ND', 'n/d', 'NA', ''], None)
        hosp_sem_leitos = col_leitos.isna().sum()
    else:
        total_leitos = 0
        col_leitos = df_hospedagem['Leitos'].replace(['-', 'ND', 'n/d', 'NA', ''], None)
        hosp_sem_leitos = col_leitos.isna().sum()

    # Média de UH por hospedagem
    media_uh = total_unidades_habitacionais/total_hospedagem
    
    # Média de Leitos por Unidade Habitacional
    media_leitos = total_leitos/total_unidades_habitacionais
    
    # Acumula os dados
    dados.append({
        'Nome_Municipio': nome,
        'Populacao': populacao,
        'Total_Restaurantes': total_restaurantes,
        'Total_Parques_Tematicos': total_parques,
        'Area_Total_Parques': area_total_parque_tematico,
        'Parques_Sem_Area': parques_sem_area,
        'Total_Empreendimentos_Lazer': total_empreendimentos_lazer,
        'Area_Total_Empreendimento_Lazer': area_total_empreendimento_lazer,
        'Empreendimentos_Lazer_Sem_Area': empreendimentos_lazer_sem_area,
        'Total_Hospedagem': total_hospedagem,
        'Unidades_Habitacionais': total_unidades_habitacionais,
        'Hospedagem_Sem_Dados_Unidades_Habitacionais': hosp_sem_uni_hab,
        'Media_Unidades_Habitacionais_por_Hospedagem': media_uh,
        'Leitos': total_leitos,
        'Hospedagem_Sem_Dados_Leitos': hosp_sem_leitos,
        'Media_Leitos_por_UH': media_leitos
    })

informacoes_municipio = pd.DataFrame(dados)

# Recalcula a coluna de restaurantes por mil habitantes
#informacoes_municipio['Restaurantes_Por_Mil_Habitantes'] = (
#    informacoes_municipio.apply(
#        lambda row: (row['Total_Restaurantes'] / row['Populacao']) * 1000
#        if pd.notnull(row['Populacao']) and row['Populacao'] > 0 else None,
#        axis=1
#    )
#)

# Meio de Hospedagem por mil habitantes
#informacoes_municipio['Hospedagem_Por_Mil_Habitantes'] = (
#    informacoes_municipio.apply(
#        lambda row: (row['Total_Hospedagem'] / row['Populacao']) * 1000
#        if pd.notnull(row['Populacao']) and row['Populacao'] > 0 else None,
#        axis=1
#    )
#)

# U.H. por 1 k.hab.
#informacoes_municipio['Unidade_Habitacional_Por_Mil_Habitantes'] = (
#    informacoes_municipio.apply(
#        lambda row: (row['Unidades_Habitacionais'] / row['Populacao']) * 1000
#        if pd.notnull(row['Populacao']) and row['Populacao'] > 0 else None,
#        axis=1
#    )
#)

# Leitos por 1 k.hab.
#informacoes_municipio['Leitos_Por_Mil_Habitantes'] = (
#    informacoes_municipio.apply(
#        lambda row: (row['Leitos'] / row['Populacao']) * 1000
#        if pd.notnull(row['Populacao']) and row['Populacao'] > 0 else None,
#        axis=1
#    )
#)

display(informacoes_municipio)

NameError: name 'dados_por_municipio' is not defined

In [76]:
informacoes_municipio.to_excel('Informações por Município.xlsx', index=False)

In [67]:
display(informacoes_municipio[informacoes_municipio['Empreendimentos_Lazer_Sem_Area'] > 0])

,Nome_Municipio,Populacao,Total_Restaurantes,Total_Parques_Tematicos,Area_Total_Parques,Parques_Sem_Area,Total_Empreendimentos_Lazer,Area_Total_Empreendimento_Lazer,Empreendimentos_Lazer_Sem_Area,Total_Hospedagem,Unidades_Habitacionais,Hospedagem_Sem_Dados_Unidades_Habitacionais,Media_Unidades_Habitacionais_por_Hospedagem,Leitos,Hospedagem_Sem_Dados_Leitos,Media_Leitos_por_UH,Restaurantes_Por_Mil_Habitantes,Hospedagem_Por_Mil_Habitantes
86,Florianópolis,537211.0,482,2,140100.0,0,7,112732.0,1,285,11876.0,0,41.670175,32267.0,0,2.716992,0.897227,0.530518
270,Treze Tílias,8787.0,12,1,73889080.0,0,5,54975.0,1,13,287.0,0,22.076923,772.0,0,2.689895,1.365654,1.479458


In [68]:
display(informacoes_municipio[informacoes_municipio['Nome_Municipio'] == '-'])

,Nome_Municipio,Populacao,Total_Restaurantes,Total_Parques_Tematicos,Area_Total_Parques,Parques_Sem_Area,Total_Empreendimentos_Lazer,Area_Total_Empreendimento_Lazer,Empreendimentos_Lazer_Sem_Area,Total_Hospedagem,Unidades_Habitacionais,Hospedagem_Sem_Dados_Unidades_Habitacionais,Media_Unidades_Habitacionais_por_Hospedagem,Leitos,Hospedagem_Sem_Dados_Leitos,Media_Leitos_por_UH,Restaurantes_Por_Mil_Habitantes,Hospedagem_Por_Mil_Habitantes
0,-,NaN,1,0,NaN,0,0,NaN,0,1,4.0,0,4.0,4.0,0,1.0,NaN,NaN


In [70]:
print(dados_por_municipio['-'])

{'meio_de_hospedagem':       Atividade Turística  Número de Inscrição do CNPJ  \
22582  Meio de Hospedagem               29536700000187   

                Nome Fantasia Situação da Atividade Tipo de Estabelecimento  \
22582  Pousada Flocos de Neve              Operação                  Matriz   

           Natureza Jurídica         Porte Endereço Completo Receita Federal  \
22582  Empresário Individual  MICROEMPRESA                        SC-439 S/N   

       UF Município  ...               E-mail Comercial Website  \
22582  SC         -  ...  pousadaflocosdeneve@gmail.com       -   

      Número do Certificado Validade do Certificado Idiomas  \
22582                     -                       -       -   

      Tipo de Hospedagem Unidade Habitacionais Leitos UHs Acessíveis  \
22582            Pousada                     4      4              0   

      Leitos Acessíveis  
22582                 0  

[1 rows x 22 columns], 'empreendimento_lazer': Empty DataFrame
Columns: [Ativida

In [71]:
informacoes_municipio['Nome_Municipio'].unique()

array(['-', 'Abdon Batista', 'Abelardo Luz', 'Agrolândia', 'Agronômica',
       'Alfredo Wagner', 'Alto Bela Vista', 'Anchieta', 'Angelina',
       'Anita Garibaldi', 'Anitápolis', 'Antônio Carlos', 'Apiúna',
       'Arabutã', 'Araquari', 'Araranguá', 'Armazém', 'Arroio Trinta',
       'Arvoredo', 'Ascurra', 'Atalanta', 'Aurora',
       'Balneário Arroio do Silva', 'Balneário Barra do Sul',
       'Balneário Camboriú', 'Balneário Gaivota', 'Balneário Piçarras',
       'Balneário Rincão', 'Bandeirante', 'Barra Bonita', 'Barra Velha',
       'Bela Vista do Toldo', 'Belmonte', 'Benedito Novo', 'Biguaçu',
       'Blumenau', 'Bocaina do Sul', 'Bom Jardim da Serra', 'Bom Jesus',
       'Bom Jesus do Oeste', 'Bom Retiro', 'Bombinhas', 'Botuverá',
       'Braço do Norte', 'Braço do Trombudo', 'Brunópolis', 'Brusque',
       'Caibi', 'Calmon', 'Camboriú', 'Campo Alegre', 'Campo Belo do Sul',
       'Campo Erê', 'Campos Novos', 'Canelinha', 'Canoinhas', 'Capinzal',
       'Capivari de Baixo', 'C

In [52]:
lista_mun = list(informacoes_municipio['Nome_Municipio'].unique())
print(len(lista_mun))

304
